In [0]:
#Upload Recommenders.py for necessary classes
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Recommenders.py to Recommenders.py
User uploaded file "Recommenders.py" with length 9674 bytes


In [0]:
import pandas as pd
import numpy as np
import time
import sqlite3
import io

import Recommenders as Recommenders
from sklearn.model_selection import train_test_split

**Load Data**

In [0]:
#Upload dataset (train_triplets.txt) to notebook
#Dataset can be found here: http://labrosa.ee.columbia.edu/millionsong/sites/default/files/challenge/train_triplets.txt.zip
from google.colab import files
uploaded = files.upload()

In [0]:
#Upload metadata set ('track_metadata.db')
!wget http://labrosa.ee.columbia.edu/millionsong/sites/default/files/AdditionalFiles/track_metadata.db

--2018-07-22 16:04:11--  http://labrosa.ee.columbia.edu/millionsong/sites/default/files/AdditionalFiles/track_metadata.db
Resolving labrosa.ee.columbia.edu (labrosa.ee.columbia.edu)... 128.59.66.11
Connecting to labrosa.ee.columbia.edu (labrosa.ee.columbia.edu)|128.59.66.11|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://labrosa.ee.columbia.edu/millionsong/sites/default/files/AdditionalFiles/track_metadata.db [following]
--2018-07-22 16:04:11--  https://labrosa.ee.columbia.edu/millionsong/sites/default/files/AdditionalFiles/track_metadata.db
Connecting to labrosa.ee.columbia.edu (labrosa.ee.columbia.edu)|128.59.66.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 746180608 (712M)
Saving to: ‘track_metadata.db’

track_metadata.db    34%[=====>              ] 244.98M  11.2MB/s    eta 43s    

track_metadata.db   100%[===================>] 711.61M  11.1MB/s    in 64s     

2018-07-22 16:05:15 (11.1 MB/s) - ‘track_metadata.db’ saved [746180608/746180608]



In [0]:
triplet_dataset = pd.read_csv(io.StringIO(uploaded['train_triplets.txt'].decode('utf-8')), nrows=10000,sep='\t', header=None, names=['user','song','play_count'])
triplet_dataset.head(n=10)

,user,song,play_count
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1
5,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOXLOQG12AF72A2D55,1
6,d7083f5e1d50c264277d624340edaaf3dc16095b,SOUVUHC12A67020E3B,1
7,d7083f5e1d50c264277d624340edaaf3dc16095b,SOUQERE12A58A75633,1
8,d7083f5e1d50c264277d624340edaaf3dc16095b,SOIPJAX12A8C141A2D,1
9,d7083f5e1d50c264277d624340edaaf3dc16095b,SOEFCDJ12AB0185FA0,2


**User and Play Counts**

In [0]:
#User and total play counts
output_dict = {}
with open('train_triplets.txt') as f:
    for line_number, line in enumerate(f):
        user = line.split('\t')[0]
        play_count = int(line.split('\t')[2])
        if user in output_dict:
            play_count +=output_dict[user]
            output_dict.update({user:play_count})
        output_dict.update({user:play_count})
output_list = [{'user':k,'play_count':v} for k,v in output_dict.items()]
play_count_df = pd.DataFrame(output_list)
play_count_df = play_count_df.sort_values(by = 'play_count', ascending = False)

In [0]:
play_count_df.to_csv(path_or_buf='user_playcount_df.csv', index = False)

**Song and Play Counts**

In [0]:
#Song and total play counts
output_dict = {}
with open('train_triplets.txt') as f:
    for line_number, line in enumerate(f):
        song = line.split('\t')[1]
        play_count = int(line.split('\t')[2])
        if song in output_dict:
            play_count +=output_dict[song]
            output_dict.update({song:play_count})
        output_dict.update({song:play_count})
output_list = [{'song':k,'play_count':v} for k,v in output_dict.items()]
song_count_df = pd.DataFrame(output_list)
song_count_df = song_count_df.sort_values(by = 'play_count', ascending = False)

In [0]:
song_count_df.to_csv(path_or_buf='song_playcount_df.csv', index = False)

In [0]:
#Top user play count
play_count_df = pd.read_csv(filepath_or_buffer='user_playcount_df.csv')
play_count_df.head(n =10)

,play_count,user
0,1192,c6150292374fb1dad89982367b3245dd5004c718
1,1119,b51eeda3c09e2426f2e52dca8444a8c0cab6265a
2,796,96deca47a75bb70478c086f435426485d575746a
3,767,a353a0a881172a15bec73b5b517f4da1d5bdcc54
4,693,44f010b69895c676486861f5e2dde09eaaf9706a
5,644,bac526d33afcdf08d4276799ffa7c72b08c4ccf4
6,638,a72d150c16341e1849c40226eb0d322cadcf050d
7,616,ae016a8529578fe7f69ab98b19838e94ded02f78
8,611,c9e5a81eb8b49cd9433de80de65242b890bf7a0b
9,608,388f7b7150bd020ce59b82603f7dd29f590ff8ed


In [0]:
#Top song play count
song_count_df = pd.read_csv(filepath_or_buffer='song_playcount_df.csv')
song_count_df.head(10)

,play_count,song
0,4065,SOBONKR12A58A7A7E0
1,3605,SOAUWYT12A81C206F1
2,2650,SOSXLTC12AF72A7F54
3,2078,SOFRQTD12A81C233C0
4,1768,SOAXGDH12A8C13F8A1
5,1704,SOEGIYH12A6D4FC0E3
6,1387,SONYKOW12AB01849C9
7,1300,SOVDSJC12A58A7A271
8,1132,SOBOUPA12A6D4F81F1
9,1129,SOPUCYA12A8C13A694


**Data Subsets** - Train/Test Split

In [0]:
total_play_count = sum(song_count_df.play_count)
(float(play_count_df.head(n=100000).play_count.sum())/total_play_count)*100
play_count_subset = play_count_df.head(n=100000)

In [0]:
(float(song_count_df.head(n=30000).play_count.sum())/total_play_count)*100

94.54334381398981

In [0]:
song_count_subset = song_count_df.head(n=30000)

In [0]:
user_subset = list(play_count_subset.user)
song_subset = list(song_count_subset.song)

In [0]:
triplet_dataset = pd.read_csv(io.StringIO(uploaded['train_triplets.txt'].decode('utf-8')), nrows=10000,sep='\t', header=None, names=['user','song','play_count'])

triplet_dataset_sub = triplet_dataset[triplet_dataset.user.isin(user_subset) ]
del(triplet_dataset)
triplet_dataset_sub_song = triplet_dataset_sub[triplet_dataset_sub.song.isin(song_subset)]
del(triplet_dataset_sub)

In [0]:
triplet_dataset_sub_song.to_csv(path_or_buf='triplet_dataset_sub_song.csv', index=False)

In [0]:
triplet_dataset_sub_song.shape

(8401, 3)

In [0]:
triplet_dataset_sub_song.head(n=10)

,user,song,play_count
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1
5,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOXLOQG12AF72A2D55,1
6,d7083f5e1d50c264277d624340edaaf3dc16095b,SOUVUHC12A67020E3B,1
8,d7083f5e1d50c264277d624340edaaf3dc16095b,SOIPJAX12A8C141A2D,1
11,d7083f5e1d50c264277d624340edaaf3dc16095b,SOZPZGN12A8C135B45,1
12,d7083f5e1d50c264277d624340edaaf3dc16095b,SOPFVWP12A6D4FC636,1


**Metadata**

In [0]:
conn = sqlite3.connect('track_metadata.db')
cur = conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
cur.fetchall()

[('songs',)]

In [0]:
track_metadata_df = pd.read_sql(con=conn, sql='select * from songs')
track_metadata_df_sub = track_metadata_df[track_metadata_df.song_id.isin(song_subset)]

In [0]:
track_metadata_df_sub.to_csv(path_or_buf='track_metadata_df_sub.csv', index=False)

In [0]:
track_metadata_df_sub.shape

(30381, 14)

In [0]:
triplet_dataset_sub_song = pd.read_csv(filepath_or_buffer='triplet_dataset_sub_song.csv')
track_metadata_df_sub = pd.read_csv(filepath_or_buffer='track_metadata_df_sub.csv')

In [0]:
del(track_metadata_df_sub['track_id'])
del(track_metadata_df_sub['artist_mbid'])
track_metadata_df_sub = track_metadata_df_sub.drop_duplicates(['song_id'])
triplet_dataset_sub_song_merged = pd.merge(triplet_dataset_sub_song, track_metadata_df_sub, how='left', left_on='song', right_on='song_id')
triplet_dataset_sub_song_merged.rename(columns={'play_count':'listen_count'},inplace=True)

In [0]:
# maybe do not del(triplet_dataset_sub_song_merged['song_id']) ??

del(triplet_dataset_sub_song_merged['artist_id'])
del(triplet_dataset_sub_song_merged['duration'])
del(triplet_dataset_sub_song_merged['artist_familiarity'])
del(triplet_dataset_sub_song_merged['artist_hotttnesss'])
del(triplet_dataset_sub_song_merged['track_7digitalid'])
del(triplet_dataset_sub_song_merged['shs_perf'])
del(triplet_dataset_sub_song_merged['shs_work'])

In [0]:
triplet_dataset_sub_song_merged.head(n=10)

,user,song,listen_count,title,song_id,release,artist_name,year
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1,You're The One,SOBONKR12A58A7A7E0,If There Was A Way,Dwight Yoakam,1990
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1,Horn Concerto No. 4 in E flat K495: II. Romanc...,SOEGIYH12A6D4FC0E3,Mozart - Eine kleine Nachtmusik,Barry Tuckwell/Academy of St Martin-in-the-Fie...,0
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1,Tive Sim,SOFLJQZ12A6D4FADA6,Nova Bis-Cartola,Cartola,1974
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1,Catch You Baby (Steve Pitron & Max Sanna Radio...,SOHTKMO12AB01843B0,Catch You Baby,Lonnie Gordon,0
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1,El Cuatrero,SODQZCY12A6D4F9D11,From Argentina To The World,Miguel Calo,2001
5,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOXLOQG12AF72A2D55,1,Unite (2009 Digital Remaster),SOXLOQG12AF72A2D55,Hello Nasty (Remastered Edition),Beastie Boys,0
6,d7083f5e1d50c264277d624340edaaf3dc16095b,SOUVUHC12A67020E3B,1,Lift Me Up,SOUVUHC12A67020E3B,Lift Me Up EP,Moby,2005
7,d7083f5e1d50c264277d624340edaaf3dc16095b,SOIPJAX12A8C141A2D,1,Open Your Eyes,SOIPJAX12A8C141A2D,Nice 90s,Guano Apes,1997
8,d7083f5e1d50c264277d624340edaaf3dc16095b,SOZPZGN12A8C135B45,1,Dreamcatcher,SOZPZGN12A8C135B45,Call of the Mystic,Bahramji & Maneesh De Moor,2005
9,d7083f5e1d50c264277d624340edaaf3dc16095b,SOPFVWP12A6D4FC636,1,Sleep Sound In Jesus,SOPFVWP12A6D4FC636,Sleep Sound In Jesus Platinum,Michael Card,0


**Matrix Factorization**

In [0]:
triplet_dataset_sub_song_merged_sum_df = triplet_dataset_sub_song_merged[['user','listen_count']].groupby('user').sum().reset_index()
triplet_dataset_sub_song_merged_sum_df.rename(columns={'listen_count':'total_listen_count'},inplace=True)
triplet_dataset_sub_song_merged = pd.merge(triplet_dataset_sub_song_merged,triplet_dataset_sub_song_merged_sum_df)
triplet_dataset_sub_song_merged['fractional_play_count'] = triplet_dataset_sub_song_merged['listen_count']/triplet_dataset_sub_song_merged['total_listen_count']

In [0]:
triplet_dataset_sub_song_merged[triplet_dataset_sub_song_merged.user =='d6589314c0a9bcbca4fee0c93b14bc402363afea'][['user','song','listen_count','fractional_play_count']].head()

,user,song,listen_count,fractional_play_count


In [0]:
from scipy.sparse import coo_matrix

small_set = triplet_dataset_sub_song_merged
user_codes = small_set.user.drop_duplicates().reset_index()
song_codes = small_set.song.drop_duplicates().reset_index()
user_codes.rename(columns={'index':'user_index'}, inplace=True)
song_codes.rename(columns={'index':'song_index'}, inplace=True)
song_codes['so_index_value'] = list(song_codes.index)
user_codes['us_index_value'] = list(user_codes.index)
small_set = pd.merge(small_set,song_codes,how='left')
small_set = pd.merge(small_set,user_codes,how='left')
mat_candidate = small_set[['us_index_value','so_index_value','fractional_play_count']]
data_array = mat_candidate.fractional_play_count.values
row_array = mat_candidate.us_index_value.values
col_array = mat_candidate.so_index_value.values

data_sparse = coo_matrix((data_array, (row_array, col_array)),dtype=float)

In [0]:
data_sparse

<752x5806 sparse matrix of type '<class 'numpy.float64'>'
	with 8401 stored elements in COOrdinate format>

In [0]:
user_codes[user_codes.user =='2a2f776cbac6df64d6cb505e7e834e01684673b6']

,user_index,user,us_index_value


**Singular-Value Decomposition - SVD**

In [0]:
import math as mt
from scipy.sparse.linalg import * #used for matrix multiplication
from scipy.sparse.linalg import svds
from scipy.sparse import csc_matrix

In [0]:
def compute_svd(urm, K):
    U, s, Vt = svds(urm, K)

    dim = (len(s), len(s))
    S = np.zeros(dim, dtype=np.float32)
    for i in range(0, len(s)):
        S[i,i] = mt.sqrt(s[i])

    U = csc_matrix(U, dtype=np.float32)
    S = csc_matrix(S, dtype=np.float32)
    Vt = csc_matrix(Vt, dtype=np.float32)
    
    return U, S, Vt

def compute_estimated_matrix(urm, U, S, Vt, uTest, K, test):
    rightTerm = S*Vt 
    max_recommendation = 250
    estimatedRatings = np.zeros(shape=(MAX_UID, MAX_PID), dtype=np.float16)
    recomendRatings = np.zeros(shape=(MAX_UID,max_recommendation ), dtype=np.float16)
    for userTest in uTest:
        prod = U[userTest, :]*rightTerm
        estimatedRatings[userTest, :] = prod.todense()
        recomendRatings[userTest, :] = (-estimatedRatings[userTest, :]).argsort()[:max_recommendation]
    return recomendRatings

In [0]:
K=50
urm = data_sparse
MAX_PID = urm.shape[1]
MAX_UID = urm.shape[0]

U, S, Vt = compute_svd(urm, K)

Specify User(s):

In [0]:
uTest = [4,5,6,7,8,10]

uTest_recommended_items = compute_estimated_matrix(urm, U, S, Vt, uTest, K, True)

Specify number of recommendations per user. Output results.

In [0]:
for user in uTest:
    print("Recommendation for user with user id {}". format(user))
    rank_value = 1
    for i in uTest_recommended_items[user,0:10]:
        song_details = small_set[small_set.so_index_value == i].drop_duplicates('so_index_value')[['title','artist_name']]
        print("The number {} recommended song is {} BY {}".format(rank_value, list(song_details['title'])[0],list(song_details['artist_name'])[0]))
        rank_value+=1

Recommendation for user with user id 4
The number 1 recommended song is Sincerité Et Jalousie BY Alliance Ethnik
The number 2 recommended song is Too Hot Ta Trot BY Commodores
The number 3 recommended song is Strut (1993 Digital Remaster) BY Sheena Easton
The number 4 recommended song is Collecting Skylines BY Swearing At Motorists
The number 5 recommended song is Hotel California BY Eagles
The number 6 recommended song is Crazy Dream BY Michael Learns To Rock
The number 7 recommended song is Fox On The Run BY Sweet
The number 8 recommended song is Fox On The Run BY Sweet
The number 9 recommended song is Fox On The Run BY Sweet
The number 10 recommended song is You'll Be In My Heart BY Phil Collins
Recommendation for user with user id 5
The number 1 recommended song is Hotel California BY Eagles
The number 2 recommended song is Dog Days Are Over (Radio Edit) BY Florence + The Machine
The number 3 recommended song is Brave The Elements BY Colossal
The number 4 recommended song is How Yo

In [0]:
uTest = [10]
#Get estimated rating for test user
print("Predictied ratings:")
uTest_recommended_items = compute_estimated_matrix(urm, U, S, Vt, uTest, K, True)

Predictied ratings:


Recommend top two songs for a specific user:

In [0]:
for user in uTest:
    print("Recommendation for user with user id {}". format(user))
    rank_value = 1
    for i in uTest_recommended_items[user,0:2]:
        song_details = small_set[small_set.so_index_value == i].drop_duplicates('so_index_value')[['title','artist_name','song_id']]
        print("The number {} recommended song is {} BY {} with song id {}".format(rank_value, list(song_details['title'])[0],list(song_details['artist_name'])[0],list(song_details['song_id'])[0]))
        rank_value+=1
        

Recommendation for user with user id 10
The number 1 recommended song is Secrets BY OneRepublic with song id SONYKOW12AB01849C9
The number 2 recommended song is Drop The World BY Lil Wayne / Eminem with song id SOUVTSM12AC468F6A7
